In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone

In [2]:
### Calculation exchange rate of reference to further math on pct_change:
def exchange_reference(df, date_reference):
    
    memory = {}
        
    for currency_code in df["currency_code"].unique():
        
        if currency_code != "VED":
            date_reference = date_reference
        else:
            date_reference = "2021-10-10"

    
        ## Select currency for calculations:
        df_exchange = df[df['currency_code'] == currency_code ]

        ## Select date to calculate reference

        df_date_reference = df_exchange.loc[(df_exchange.date).dt.date == pd.Timestamp(date_reference)]

        ### Averare exchange rate for day of reference:

        implicit_exchange_reference = df_date_reference.implicit_exchange.mean()
        
        memory[currency_code] = implicit_exchange_reference

    ### Percentual Variation

#     equation = ((df_exchange.implicit_exchange / implicit_exchange_reference)-1)*100
    
    
    return memory

In [3]:
from sqlalchemy import Column, Integer, Float, String, DateTime,Boolean, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

path_to_currencies = "sqlite:///data/CurrenciesDataBase.db"
df_market = pd.read_sql('currencies_vs_btc', path_to_currencies)

### The pct calculations are not properly saved on the dataset unless I upload one previously saved. Why?
# The pipeline defined solve the issue by saving the calculated data on a new dataset but the calculations
# are done in the full dataset everytime the code runs.
# This practice is not scalable

# The pct could be calculated on the web interface for a given date selected by the users but this implies 
# more processing time

In [ ]:
df = df_market.groupby(by=["date","currency_code"]).mean()
df.reset_index(inplace=True)

### Dataset with implicit exchange of reference by currency:
date_reference = "2021-10-03"

exchange = exchange_reference(df, date_reference)
exchange = pd.DataFrame.from_dict(exchange, orient="index")

df['pct'] = df.apply(lambda x : ((x['implicit_exchange']/exchange.loc[x["currency_code"]])-1)*100, axis = 1 )

### Drop Null values:

df.dropna(subset=['pct'], inplace=True)

In [4]:
# df.info()

In [ ]:
# ### Saving on DB Version 2

# engine = create_engine('sqlite:///data/CurrenciesDataBase_V2.db')
# print(type(engine))
# df.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

In [ ]:
now = dt.datetime.now()
print("Last Execution: ")
print(str(now))
print('Completed')